Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


Or use OneHotEncoder from sklearn.preprocessing

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
encoder = OneHotEncoder(categories='auto')
train_1hot_labels = encoder.fit_transform(train_labels.reshape(-1,1)).toarray()
train_1hot_labels.shape

(2000000, 2)

We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [7]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

W0905 15:20:18.544546 10312 deprecation.py:323] From <ipython-input-7-38724ffcbbe8>:30: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Let's run this computation and iterate:

In [8]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
          # Calling .eval() on valid_prediction is basically like calling run(), but
          # just to get that one numpy array. Note that it recomputes all its graph
          # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            print('-' * 30)
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 15.607200
Training accuracy: 10.8%
Validation accuracy: 14.7%
------------------------------
Loss at step 100: 2.429701
Training accuracy: 71.0%
Validation accuracy: 69.8%
------------------------------
Loss at step 200: 1.966360
Training accuracy: 74.0%
Validation accuracy: 72.2%
------------------------------
Loss at step 300: 1.713631
Training accuracy: 75.3%
Validation accuracy: 73.1%
------------------------------
Loss at step 400: 1.541352
Training accuracy: 76.0%
Validation accuracy: 73.6%
------------------------------
Loss at step 500: 1.412661
Training accuracy: 76.5%
Validation accuracy: 74.2%
------------------------------
Loss at step 600: 1.311719
Training accuracy: 77.0%
Validation accuracy: 74.3%
------------------------------
Loss at step 700: 1.229626
Training accuracy: 77.6%
Validation accuracy: 74.6%
------------------------------
Loss at step 800: 1.160997
Training accuracy: 78.1%
Validation accuracy: 74.8%
------------------------------

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [9]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print('-' * 30)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.383240
Minibatch accuracy: 10.9%
Validation accuracy: 13.7%
------------------------------
Minibatch loss at step 500: 2.265983
Minibatch accuracy: 71.9%
Validation accuracy: 75.3%
------------------------------
Minibatch loss at step 1000: 1.489524
Minibatch accuracy: 74.2%
Validation accuracy: 76.8%
------------------------------
Minibatch loss at step 1500: 1.072389
Minibatch accuracy: 78.9%
Validation accuracy: 78.1%
------------------------------
Minibatch loss at step 2000: 1.131181
Minibatch accuracy: 79.7%
Validation accuracy: 76.7%
------------------------------
Minibatch loss at step 2500: 1.140712
Minibatch accuracy: 75.8%
Validation accuracy: 78.5%
------------------------------
Minibatch loss at step 3000: 0.873922
Minibatch accuracy: 78.1%
Validation accuracy: 78.3%
------------------------------
Test accuracy: 86.2%


In [11]:
reset_graph()

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='sgd',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
model.fit(train_dataset, train_labels, batch_size=128, validation_data = (valid_dataset, valid_labels))

W0905 15:20:38.318546 10312 deprecation.py:506] From C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 200000 samples, validate on 10000 samples
200000/200000 [==============================] - 2s 12us/sample - loss: 0.8573 - acc: 0.7780 - val_loss: 0.7232 - val_acc: 0.8132


In [12]:
model.evaluate(test_dataset, test_labels)

10000/10000 [==============================] - 0s 19us/sample - loss: 0.4844 - acc: 0.8803


[0.4843584372520447, 0.8803]

In [13]:
model.save('my_logistic_model.h5')

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [14]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc', 0)>.95):
                print("\nReached 95% accuracy so cancelling training!")
                self.model.stop_training = True
        
callback = myCallback()

In [15]:
def train_mnist(x_train, y_train):

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='sgd',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(x_train, y_train, epochs=10, callbacks=[callback])
    return model, history

In [16]:
model, history = train_mnist(train_dataset.reshape(-1, 784), train_labels)

Epoch 1/10
200000/200000 [==============================] - 27s 134us/sample - loss: 0.6009 - acc: 0.8314
Epoch 2/10
200000/200000 [==============================] - 28s 141us/sample - loss: 0.4878 - acc: 0.8598
Epoch 3/10
200000/200000 [==============================] - 29s 143us/sample - loss: 0.4457 - acc: 0.8712
Epoch 4/10
200000/200000 [==============================] - 26s 130us/sample - loss: 0.4149 - acc: 0.8806
Epoch 5/10
200000/200000 [==============================] - 29s 144us/sample - loss: 0.3910 - acc: 0.8876
Epoch 6/10
200000/200000 [==============================] - 30s 149us/sample - loss: 0.3712 - acc: 0.8930
Epoch 7/10
200000/200000 [==============================] - 27s 136us/sample - loss: 0.3548 - acc: 0.8983
Epoch 8/10
200000/200000 [==============================] - 27s 134us/sample - loss: 0.3401 - acc: 0.9029
Epoch 9/10
200000/200000 [==============================] - 30s 151us/sample - loss: 0.3269 - acc: 0.9064
Epoch 10/10
200000/200000 [===================

In [17]:
print('Test accuracy', model.evaluate(test_dataset, test_labels, callbacks=[callback])[1]*100, '%')

10000/10000 [==============================] - 1s 102us/sample - loss: 0.1745 - acc: 0.9517
Test accuracy 95.169997215271 %


In [18]:
model.save('my_nn_model.h5')

In [19]:
new_model = tf.keras.models.load_model('my_nn_model.h5')
new_model.summary()

W0905 15:25:25.390594 10312 deprecation.py:506] From C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0905 15:25:25.392595 10312 deprecation.py:506] From C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0905 15:25:25.522594 10312 hdf5_format.py:263] Sequential models without an `input_shape` passed to the first layer cannot reload their optimizer state. As a result, your model isstarting with a fres

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  803840    
_________________________________________________________________
dense_2 (Dense)              multiple                  10250     
Total params: 814,090
Trainable params: 814,090
Non-trainable params: 0
_________________________________________________________________


In [20]:
[x.shape for x in new_model.get_weights()]

[(784, 1024), (1024,), (1024, 10), (10,)]